# Etude sur les joueurs de ligue 1

#Recuperation des données

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3
import numpy as np
from numpy import random

In [2]:
#load data (make sure you have downloaded database.sqlite)
with sqlite3.connect('C:/Users/bambac/Desktop/data science/football/database.sqlite') as con:
    countries = pd.read_sql_query("SELECT * from Country", con)
    matches = pd.read_sql_query("SELECT * from Match", con)
    leagues = pd.read_sql_query("SELECT * from League", con)
    teams = pd.read_sql_query("SELECT * from Team", con)
    Players = pd.read_sql_query("SELECT * from Player", con)
    PlayerStats = pd.read_sql_query("SELECT * from Player_Stats", con)

In [3]:
#CREATION DE LA TABLE DES JOUEUR     
result=pd.DataFrame({'season':[],
                     'country_id':[],    
                     'player_api_id':[]})

for i in range(1,12) :
      temp=pd.DataFrame({'season' : matches['season'],
                   'country_id' : matches['country_id'],
                   'player_api_id' : matches['home_player_'+str(i)].fillna(0).astype(int)
                   })
      temp2=pd.DataFrame({'season' : matches['season'],
                   'country_id' : matches['country_id'],
                   'player_api_id' : matches['away_player_'+str(i)].fillna(0).astype(int)
                   })
      result=result.append(temp, ignore_index=True)
      result=result.append(temp2, ignore_index=True)
      
result.country_id=result.country_id.fillna(0).astype(int)
result.player_api_id=result.player_api_id.fillna(0).astype(int)
result.tail()

,country_id,player_api_id,season
570785,24524,289732,2015/2016
570786,24524,95257,2015/2016
570787,24524,275122,2015/2016
570788,24524,194532,2015/2016
570789,24524,527103,2015/2016


In [5]:
#Agregation par season league et joueur
result['nb_match']=1
grouped=result.groupby(['country_id','season','player_api_id'])
agg = grouped.sum()
joueur = agg.reset_index()

In [6]:
# ajout des informations 
joueur=pd.merge(joueur,leagues,how='inner',on='country_id')
joueur=pd.merge(joueur,Players,how='inner',on='player_api_id')
joueur.head(10)
joueur['age']=joueur['season'].str[:4].astype(int)-joueur['birthday'].str[:4].astype(int)

In [8]:
#aggergation par joueur 
grouped=joueur.groupby(['name','season']).mean().reset_index()

In [9]:
for i in grouped.name.unique() :
    temp=grouped[grouped.name==i]
    objects = temp.season
    y_pos = np.arange(len(objects))
    performance = temp.age
    plt.bar(y_pos, performance, align='center', alpha=0.5)
    plt.xticks(y_pos, objects)
    plt.ylabel('age moyen')
    plt.title('Evolution de l age moyen en '+i)
    plt.show()

In [10]:
joueur.head()

,country_id,season,player_api_id,nb_match,id_x,name,id_y,player_name,player_fifa_api_id,birthday,height,weight,age
0,1,2008/2009,6800,4,1,Belgium Jupiler League,5144,Jonas Ivens,136110,1984-10-14 00:00:00,187.96,190,24
1,1,2009/2010,6800,24,1,Belgium Jupiler League,5144,Jonas Ivens,136110,1984-10-14 00:00:00,187.96,190,25
2,1,2012/2013,6800,8,1,Belgium Jupiler League,5144,Jonas Ivens,136110,1984-10-14 00:00:00,187.96,190,28
3,13240,2010/2011,6800,33,13240,Netherlands Eredivisie,5144,Jonas Ivens,136110,1984-10-14 00:00:00,187.96,190,26
4,13240,2011/2012,6800,19,13240,Netherlands Eredivisie,5144,Jonas Ivens,136110,1984-10-14 00:00:00,187.96,190,27


In [13]:
PlayerStats['annee']=PlayerStats['date_stat'].str[:4].astype(int)
PlayerStats.head()

,id,player_fifa_api_id,player_api_id,date_stat,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,annee
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0,2016
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0,2015
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0,2015
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0,2015
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0,2007
